# Multi‑turn Weather Chat
Run this within the virtual environment **(env_ollama)**!

In [1]:
!which python

/home/matthias/Desktop/MachineLearning/Ollama_Udemy/env_ollama/bin/python


This notebook enables a **multi‑turn chat** about the **current weather** in multiple cities.

- The script `open-meteo-api.py` can retrieve weather data from Open‑Meteo (but only for **Konstanz**, **Berlin** and **Munich**).
- The notebook uses that `open-meteo-api.py` script **API** (`get_current_weather`) to fetch (almost) live weather for one of the predefined cities mentioned above.

Requirements: `ollama serve` running locally, and a model like `llama3.2` pulled.

In [2]:
import json
import runpy
import ollama
import pandas as pd
from pathlib import Path
# This will execute the script once; we capture its globals and reuse the configured client.
open_meteo_globals = runpy.run_path(str(Path.cwd() / "../../Scripts/open-meteo-api.py"))
openmeteo = open_meteo_globals["openmeteo"]
# Define cities and their coordinates
CITIES = {
    "berlin":   {"name": "Berlin, DE",   "lat": 52.5200, "lon": 13.4050},
    "konstanz": {"name": "Konstanz, DE", "lat": 47.6780, "lon": 9.1737},
    "munich":   {"name": "Munich, DE",   "lat": 48.1374, "lon": 11.5755},
}
def list_cities() -> str:
    return ", ".join([f"{k} ({v['name']})" for k, v in CITIES.items()])
# Weather tool: current temperature etc. (Open‑Meteo, no API key) ---
def get_current_weather(city: str) -> dict:
    """Get current weather for a predefined city key (berlin/konstanz/munich)."""
    if not city:
        raise ValueError(f"Please specify a city. Options: {list_cities()}")
    key = city.strip().lower()
    if key not in CITIES:
        raise ValueError(f"Unknown city '{city}'. Choose one of: {list_cities()}")
    loc = CITIES[key]
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": loc["lat"],
        "longitude": loc["lon"],
        # 'current' gives a snapshot around 'now'
        "current": ",".join([
            "temperature_2m",
            "apparent_temperature",
            "relative_humidity_2m",
            "precipitation",
            "wind_speed_10m",
        ]),
        "timezone": "UTC",
    }
    response = openmeteo.weather_api(url, params=params)[0]
    current = response.Current()
    values = [current.Variables(i).Value() for i in range(current.VariablesLength())]
    fields = ["temperature_2m", "apparent_temperature", "relative_humidity_2m", "precipitation", "wind_speed_10m"]
    payload = {fields[i]: values[i] for i in range(len(fields))}
    # Time is unix seconds in UTC
    time_utc = pd.to_datetime(current.Time(), unit="s", utc=True).isoformat()
    # Return structured data
    return {
        "city_key": key,
        "city_name": loc["name"],
        "time_utc": time_utc,
        "temperature_c": payload["temperature_2m"],
        "feels_like_c": payload["apparent_temperature"],
        "humidity_percent": payload["relative_humidity_2m"],
        "precip_mm": payload["precipitation"],
        "wind_kmh": payload["wind_speed_10m"],
    }
# Expose the tool to Ollama (function calling) ---
TOOLS = [{
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get current weather for a predefined city key.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": f"City key. One of: {', '.join(CITIES.keys())}. If user doesn't specify, ask them to choose.",
                }
            },
            "required": ["city"],
        },
    },
}]
def _run_tool_call(name: str, arguments: dict) -> str:
    if name != "get_current_weather":
        raise ValueError(f"Unknown tool: {name}")
    return json.dumps(get_current_weather(**arguments), ensure_ascii=False)
# Test loading the tool and listing cities
print("Loaded. Cities:", list_cities())

-4.6°C at 2026-01-22 UTC
Loaded. Cities: berlin (Berlin, DE), konstanz (Konstanz, DE), munich (Munich, DE)


## Chat loop

Run the cell below and chat. Examples:

- “What’s the weather in Berlin right now?”
- “Compare Konstanz and Munich.”
- “Is it raining?” (it will ask which city)


In [3]:
def chat_weather(model: str = "llama3.2"):
    system = (
        "You are a helpful weather assistant. "
        "When you need live weather, call the get_current_weather tool. "
        f"If the user doesn't specify a city, ask them to choose one of: {list_cities()}."
    )
    messages = [{"role": "system", "content": system}]
    print("Weather chat ready. Type 'cities' to see options, 'quit' to exit.\n")
    while True:
        user = input("You: ").strip()
        if not user:
            continue
        if user.lower() in {"quit", "exit"}:
            print("Bye!")
            return
        if user.lower() in {"cities", "city", "locations", "locs"}:
            print("Cities:", list_cities(), "\n")
            continue
        messages.append({"role": "user", "content": user})
        # First call: model may request tool usage
        resp = ollama.chat(model=model, messages=messages, tools=TOOLS)
        msg = resp.message
        tool_calls = getattr(msg, "tool_calls", None)
        if tool_calls:
            # Add assistant message that contains tool calls
            messages.append({"role": "assistant", "content": msg.content or "", "tool_calls": tool_calls})
            # Execute each tool call and append tool results
            for call in tool_calls:
                fn = call.get("function", {})
                name = fn.get("name")
                args = fn.get("arguments") or {}
                if isinstance(args, str):
                    args = json.loads(args)
                tool_result = _run_tool_call(name, args)
                messages.append({"role": "tool", "name": name, "content": tool_result})
            # Second call: model produces final answer grounded in tool outputs
            resp2 = ollama.chat(model=model, messages=messages)
            answer = resp2.message.content
            print(f"Assistant: {answer}\n")
            messages.append({"role": "assistant", "content": answer})
        else:
            answer = msg.content
            print(f"Assistant: {answer}\n")
            messages.append({"role": "assistant", "content": answer})
# Start:
chat_weather("llama3.2")

Weather chat ready. Type 'cities' to see options, 'quit' to exit.



You:  cities


Cities: berlin (Berlin, DE), konstanz (Konstanz, DE), munich (Munich, DE) 



You:  What is the weather like in Berlin?


Assistant: The current weather in Berlin is:

* Temperature: 3.85°C
* Feels like: 0.81°C
* Humidity: 85%
* Precipitation: 0 mm
* Wind speed: 8 km/h

Please note that these values are accurate as of the last update and may have changed since then.



...

Assistant: Here's a comparison of the current weather in Berlin, Munich, and Konstanz:

* Temperature:
	+ Berlin: 3.85°C
	+ Munich: 1.55°C (almost 2°C lower than Berlin)
	+ Konstanz: 0.63°C (about 3.22°C lower than Berlin)
* Feels like:
	+ Berlin: 0.81°C
	+ Munich: -1.74°C (very cold)
	+ Konstanz: -2.11°C (even colder than Munich)
* Humidity:
	+ Berlin: 85%
	+ Munich: 79%
	+ Konstanz: 99% (very humid)
* Precipitation:
	+ All three cities have 0 mm of precipitation
* Wind speed:
	+ Berlin: 7.99 km/h
	+ Munich: 6.29 km/h
	+ Konstanz: 4.80 km/h (the calmest)

Konstanz has the lowest temperature and feels like the coldest among the three cities, due to its high humidity.



You:  Thank you. Goodbye!


Assistant: It was nice assisting you with the weather information. Have a great day! Goodbye!



You:  quit


Bye!


Enter **quit** to exit the chat loop.

$\checkmark$